In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Lambda
from keras.layers import Concatenate
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
from keras.callbacks import ModelCheckpoint

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 90
l = 20
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Reshaping and scaling the images
x_train = x_train.reshape(x_train.shape[0],32,32,3)
x_test = x_test.reshape(x_test.shape[0],32,32,3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

train_size = x_train.shape[0]
test_size = x_test.shape[0]

In [8]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

augment_size = 5000
image_generator = ImageDataGenerator(rotation_range=10,zoom_range=0.05,width_shift_range=0.07,
                           height_shift_range=0.07,horizontal_flip=True,vertical_flip=False,
                           data_format="channels_last",zca_whitening=True)

image_generator.fit(x_train,augment=True)

/usr/local/lib/python3.6/dist-packages/keras/preprocessing/image.py:645: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
# Randomly choosing images and copying
randidx = np.random.randint(train_size,size=augment_size)
x_augmented = x_train[randidx].copy()
y_augmented = y_train[randidx].copy()

In [10]:
print(x_augmented.shape)
print(y_augmented.shape)

(5000, 32, 32, 3)
(5000, 10)


In [0]:
# Performing Image augmentation
x_augmented = image_generator.flow(x_augmented,np.zeros(augment_size),
                                  batch_size=augment_size,shuffle=False).next()[0]

In [12]:
print(x_augmented.shape)
print(y_augmented.shape)

(5000, 32, 32, 3)
(5000, 10)


In [0]:
x_train = np.concatenate((x_train,x_augmented))
y_train = np.concatenate((y_train,y_augmented))

In [14]:
print(x_train.shape)
print(y_train.shape)

(55000, 32, 32, 3)
(55000, 10)


In [15]:
trainsize = x_train.shape[0]
print(trainsize)

55000


In [0]:
# Shuffling the images
indices = np.random.permutation(trainsize)
#print(len(indices))
x_train = x_train[indices]
y_train = y_train[indices]

In [17]:
print(x_train.shape)
print(y_train.shape)

(55000, 32, 32, 3)
(55000, 10)


In [0]:
# Dense Block
# Convolution - Batch Normalization - Activation
def add_denseblock(input, num_filter = 12, dropout_rate = 0):
    global compression
    temp = input
    for _ in range(l):
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(temp)
        BatchNorm = BatchNormalization()(Conv2D_3_3)
        LRelu = LeakyReLU(alpha=0.01)(BatchNorm)
        if dropout_rate>0:
          Conv2D_3_3_drop = Dropout(dropout_rate)(LRelu)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3_drop])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0):
    global compression
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(input)
    BatchNorm = BatchNormalization()(Conv2D_BottleNeck)
    LRelu = LeakyReLU(alpha=0.01)(BatchNorm)
    if dropout_rate>0:
      Conv2D_BottleNeck_drop = Dropout(dropout_rate)(LRelu)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck_drop)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    LRelu = LeakyReLU(alpha=0.01)(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(LRelu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
# For reducing the image resolution 
import tensorflow as tf
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 20
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

skip_connection = Third_Transition

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same',name='skip_conv', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='skip_batch')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

Fifth_Block = concatenate([skip_connection, Fourth_Transition])

Sixth_Block = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(Fifth_Block)
Sixth_Block = BatchNormalization(name='final_batch')(Sixth_Block)
Sixth_Block = LeakyReLU(alpha=0.1)(Sixth_Block)

output = output_layer(Sixth_Block)

In [23]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 6)    648         conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 6)    24          conv2d_2[0][0]                   
__________________________________________________________________________________________________
leaky_re_l

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='sgd'
              ,metrics=['accuracy'])

In [25]:
# For retrieving the saved best results
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
!wget https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
from clr_callback import *

--2018-10-27 06:15:13--  https://raw.githubusercontent.com/bckenstler/CLR/master/clr_callback.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5326 (5.2K) [text/plain]
Saving to: ‘clr_callback.py.2’

clr_callback.py.2   100%[===================>]   5.20K  --.-KB/s    in 0s      

2018-10-27 06:15:13 (63.6 MB/s) - ‘clr_callback.py.2’ saved [5326/5326]



In [0]:
# Defining checkpoints
filepath="/content/gdrive/My Drive/Raajesh-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
clr_t = CyclicLR(mode='triangular',base_lr=0.0075,max_lr=0.001,step_size=1000)
callbacks_list.append(clr_t)

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

#Copied the logs for precation

Train on 55000 samples, validate on 10000 samples
Epoch 1/10
55000/55000 [==============================] - 254s 5ms/step - loss: 1.8403 - acc: 0.3271 - val_loss: 1.6418 - val_acc: 0.4305

Epoch 00001: val_acc improved from -inf to 0.43050, saving model to /content/gdrive/My Drive/Raajesh-01-0.43.hdf5
Epoch 2/10
55000/55000 [==============================] - 342s 6ms/step - loss: 1.4918 - acc: 0.4545 - val_loss: 1.8903 - val_acc: 0.4044

Epoch 00002: val_acc did not improve from 0.43050
Epoch 3/10
55000/55000 [==============================] - 224s 4ms/step - loss: 1.2990 - acc: 0.5286 - val_loss: 1.1259 - val_acc: 0.5897

Epoch 00003: val_acc improved from 0.43050 to 0.58970, saving model to /content/gdrive/My Drive/Raajesh-03-0.59.hdf5
Epoch 4/10
55000/55000 [==============================] - 382s 7ms/step - loss: 1.1469 - acc: 0.5874 - val_loss: 1.0640 - val_acc: 0.6222

Epoch 00004: val_acc improved from 0.58970 to 0.62220, saving model to /content/gdrive/My Drive/Raajesh-04-0.62.hdf5
Epoch 5/10
55000/55000 [==============================] - 323s 6ms/step - loss: 1.0539 - acc: 0.6205 - val_loss: 0.9582 - val_acc: 0.6599

Epoch 00005: val_acc improved from 0.62220 to 0.65990, saving model to /content/gdrive/My Drive/Raajesh-05-0.66.hdf5
Epoch 6/10
55000/55000 [==============================] - 391s 7ms/step - loss: 1.0224 - acc: 0.6348 - val_loss: 1.1768 - val_acc: 0.6092

Epoch 00006: val_acc did not improve from 0.65990
Epoch 7/10
55000/55000 [==============================] - 390s 7ms/step - loss: 0.9978 - acc: 0.6437 - val_loss: 1.0590 - val_acc: 0.6298

Epoch 00007: val_acc did not improve from 0.65990
Epoch 8/10
55000/55000 [==============================] - 388s 7ms/step - loss: 0.9482 - acc: 0.6587 - val_loss: 1.0357 - val_acc: 0.6341

Epoch 00008: val_acc did not improve from 0.65990
Epoch 9/10
55000/55000 [==============================] - 387s 7ms/step - loss: 0.8801 - acc: 0.6841 - val_loss: 0.8457 - val_acc: 0.6968

Epoch 00009: val_acc improved from 0.65990 to 0.69680, saving model to /content/gdrive/My Drive/Raajesh-09-0.70.hdf5
Epoch 10/10
55000/55000 [==============================] - 391s 7ms/step - loss: 0.8407 - acc: 0.6975 - val_loss: 0.8435 - val_acc: 0.7044

Epoch 00010: val_acc improved from 0.69680 to 0.70440, saving model to /content/gdrive/My Drive/Raajesh-10-0.70.hdf5
<keras.callbacks.History at 0x7f844467ed68>

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 55000 samples, validate on 10000 samples
Epoch 1/50
55000/55000 [==============================] - 219s 4ms/step - loss: 0.8360 - acc: 0.7020 - val_loss: 0.8303 - val_acc: 0.7115

Epoch 00001: val_acc improved from -inf to 0.71150, saving model to /content/gdrive/My Drive/Raajesh-01-0.71.hdf5
Epoch 2/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.8387 - acc: 0.7019 - val_loss: 0.9722 - val_acc: 0.6606

Epoch 00002: val_acc did not improve from 0.71150
Epoch 3/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.8230 - acc: 0.7045 - val_loss: 0.8155 - val_acc: 0.7157

Epoch 00003: val_acc improved from 0.71150 to 0.71570, saving model to /content/gdrive/My Drive/Raajesh-03-0.72.hdf5
Epoch 4/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.7743 - acc: 0.7227 - val_loss: 0.7608 - val_acc: 0.7292

Epoch 00004: val_acc improved from 0.71570 to 0.72920, saving model to /content/gdrive/My Drive/Raajesh-04-0.73.hdf5
Epoch 5/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.7275 - acc: 0.7419 - val_loss: 0.7262 - val_acc: 0.7463

Epoch 00005: val_acc improved from 0.72920 to 0.74630, saving model to /content/gdrive/My Drive/Raajesh-05-0.75.hdf5
Epoch 6/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.7218 - acc: 0.7437 - val_loss: 0.7312 - val_acc: 0.7469

Epoch 00006: val_acc improved from 0.74630 to 0.74690, saving model to /content/gdrive/My Drive/Raajesh-06-0.75.hdf5
Epoch 7/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.7311 - acc: 0.7401 - val_loss: 0.9977 - val_acc: 0.6750

Epoch 00007: val_acc did not improve from 0.74690
Epoch 8/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.7154 - acc: 0.7452 - val_loss: 0.7209 - val_acc: 0.7466

Epoch 00008: val_acc did not improve from 0.74690
Epoch 9/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.6659 - acc: 0.7644 - val_loss: 0.6958 - val_acc: 0.7566

Epoch 00009: val_acc improved from 0.74690 to 0.75660, saving model to /content/gdrive/My Drive/Raajesh-09-0.76.hdf5
Epoch 10/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.6374 - acc: 0.7735 - val_loss: 0.7405 - val_acc: 0.7458

Epoch 00010: val_acc did not improve from 0.75660
Epoch 11/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.6494 - acc: 0.7698 - val_loss: 0.7199 - val_acc: 0.7537

Epoch 00011: val_acc did not improve from 0.75660
Epoch 12/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.6563 - acc: 0.7662 - val_loss: 0.7112 - val_acc: 0.7531

Epoch 00012: val_acc did not improve from 0.75660
Epoch 13/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.6319 - acc: 0.7738 - val_loss: 0.6636 - val_acc: 0.7733

Epoch 00013: val_acc improved from 0.75660 to 0.77330, saving model to /content/gdrive/My Drive/Raajesh-13-0.77.hdf5
Epoch 14/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5896 - acc: 0.7913 - val_loss: 0.6245 - val_acc: 0.7799

Epoch 00014: val_acc improved from 0.77330 to 0.77990, saving model to /content/gdrive/My Drive/Raajesh-14-0.78.hdf5
Epoch 15/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.5743 - acc: 0.7945 - val_loss: 0.6688 - val_acc: 0.7696

Epoch 00015: val_acc did not improve from 0.77990
Epoch 16/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5875 - acc: 0.7929 - val_loss: 0.7003 - val_acc: 0.7614

Epoch 00016: val_acc did not improve from 0.77990
Epoch 17/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5982 - acc: 0.7861 - val_loss: 0.6680 - val_acc: 0.7726

Epoch 00017: val_acc did not improve from 0.77990
Epoch 18/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.5556 - acc: 0.8039 - val_loss: 0.6378 - val_acc: 0.7819

Epoch 00018: val_acc improved from 0.77990 to 0.78190, saving model to /content/gdrive/My Drive/Raajesh-18-0.78.hdf5
Epoch 19/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5213 - acc: 0.8162 - val_loss: 0.6162 - val_acc: 0.7915

Epoch 00019: val_acc improved from 0.78190 to 0.79150, saving model to /content/gdrive/My Drive/Raajesh-19-0.79.hdf5
Epoch 20/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5213 - acc: 0.8161 - val_loss: 0.6825 - val_acc: 0.7733

Epoch 00020: val_acc did not improve from 0.79150
Epoch 21/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5500 - acc: 0.8041 - val_loss: 0.7746 - val_acc: 0.7416

Epoch 00021: val_acc did not improve from 0.79150
Epoch 22/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.5390 - acc: 0.8072 - val_loss: 0.6523 - val_acc: 0.7821

Epoch 00022: val_acc did not improve from 0.79150
Epoch 23/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.4967 - acc: 0.8233 - val_loss: 0.6114 - val_acc: 0.7905

Epoch 00023: val_acc did not improve from 0.79150
Epoch 24/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4721 - acc: 0.8317 - val_loss: 0.6552 - val_acc: 0.7825

Epoch 00024: val_acc did not improve from 0.79150
Epoch 25/50
54912/55000 [============================>.] - ETA: 0s - loss: 0.4873 - acc: 0.8248
Epoch 00025: val_acc did not improve from 0.79150
Epoch 26/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.5092 - acc: 0.8178 - val_loss: 0.7065 - val_acc: 0.7699

Epoch 00026: val_acc did not improve from 0.79150
Epoch 27/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4803 - acc: 0.8286 - val_loss: 0.6195 - val_acc: 0.7968

Epoch 00027: val_acc improved from 0.79150 to 0.79680, saving model to /content/gdrive/My Drive/Raajesh-27-0.80.hdf5
Epoch 28/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4420 - acc: 0.8418 - val_loss: 0.6250 - val_acc: 0.7926

Epoch 00028: val_acc did not improve from 0.79680
Epoch 29/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.4341 - acc: 0.8449 - val_loss: 0.6281 - val_acc: 0.7955

Epoch 00029: val_acc did not improve from 0.79680
Epoch 30/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4642 - acc: 0.8341 - val_loss: 0.7757 - val_acc: 0.7525

Epoch 00030: val_acc did not improve from 0.79680
Epoch 31/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.4754 - acc: 0.8302 - val_loss: 0.6725 - val_acc: 0.7807

Epoch 00031: val_acc did not improve from 0.79680
Epoch 32/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4322 - acc: 0.8461 - val_loss: 0.6159 - val_acc: 0.7979

Epoch 00032: val_acc improved from 0.79680 to 0.79790, saving model to /content/gdrive/My Drive/Raajesh-32-0.80.hdf5
Epoch 33/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4039 - acc: 0.8561 - val_loss: 0.6283 - val_acc: 0.7939

Epoch 00033: val_acc did not improve from 0.79790
Epoch 34/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4086 - acc: 0.8533 - val_loss: 0.6607 - val_acc: 0.7917

Epoch 00034: val_acc did not improve from 0.79790
Epoch 35/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4393 - acc: 0.8411 - val_loss: 0.6491 - val_acc: 0.7936

Epoch 00035: val_acc did not improve from 0.79790
Epoch 36/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.4301 - acc: 0.8459 - val_loss: 0.6331 - val_acc: 0.7936

Epoch 00036: val_acc did not improve from 0.79790
Epoch 37/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.3887 - acc: 0.8590 - val_loss: 0.6120 - val_acc: 0.7999

Epoch 00037: val_acc improved from 0.79790 to 0.79990, saving model to /content/gdrive/My Drive/Raajesh-37-0.80.hdf5
Epoch 38/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3688 - acc: 0.8692 - val_loss: 0.6159 - val_acc: 0.8024

Epoch 00038: val_acc improved from 0.79990 to 0.80240, saving model to /content/gdrive/My Drive/Raajesh-38-0.80.hdf5
Epoch 39/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3872 - acc: 0.8605 - val_loss: 0.7230 - val_acc: 0.7650

Epoch 00039: val_acc did not improve from 0.80240
Epoch 40/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.4219 - acc: 0.8481 - val_loss: 0.7152 - val_acc: 0.7739

Epoch 00040: val_acc did not improve from 0.80240
Epoch 41/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3912 - acc: 0.8594 - val_loss: 0.6190 - val_acc: 0.7986

Epoch 00041: val_acc did not improve from 0.80240
Epoch 42/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3503 - acc: 0.8758 - val_loss: 0.6022 - val_acc: 0.8079

Epoch 00042: val_acc improved from 0.80240 to 0.80790, saving model to /content/gdrive/My Drive/Raajesh-42-0.81.hdf5
Epoch 43/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3466 - acc: 0.8767 - val_loss: 0.6910 - val_acc: 0.7859

Epoch 00043: val_acc did not improve from 0.80790
Epoch 44/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3734 - acc: 0.8657 - val_loss: 0.7473 - val_acc: 0.7600

Epoch 00044: val_acc did not improve from 0.80790
Epoch 45/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.3856 - acc: 0.8602 - val_loss: 0.6855 - val_acc: 0.7948

Epoch 00045: val_acc did not improve from 0.80790
Epoch 46/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3499 - acc: 0.8737 - val_loss: 0.6296 - val_acc: 0.8015

Epoch 00046: val_acc did not improve from 0.80790
Epoch 47/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3187 - acc: 0.8851 - val_loss: 0.6479 - val_acc: 0.7974

Epoch 00047: val_acc did not improve from 0.80790
Epoch 48/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3259 - acc: 0.8828 - val_loss: 0.7038 - val_acc: 0.7794

Epoch 00048: val_acc did not improve from 0.80790
Epoch 49/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3612 - acc: 0.8719 - val_loss: 0.7163 - val_acc: 0.7783

Epoch 00049: val_acc did not improve from 0.80790
Epoch 50/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.3542 - acc: 0.8729 - val_loss: 0.6653 - val_acc: 0.7909

Epoch 00050: val_acc did not improve from 0.80790
<keras.callbacks.History at 0x7f5593a87a58>

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 55000 samples, validate on 10000 samples
Epoch 1/50
55000/55000 [==============================] - 216s 4ms/step - loss: 0.3017 - acc: 0.8985 - val_loss: 0.5689 - val_acc: 0.8210

Epoch 00001: val_acc improved from -inf to 0.82100, saving model to /content/gdrive/My Drive/Raajesh-01-0.82.hdf5
Epoch 2/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2959 - acc: 0.8985 - val_loss: 0.5702 - val_acc: 0.8216

Epoch 00002: val_acc improved from 0.82100 to 0.82160, saving model to /content/gdrive/My Drive/Raajesh-02-0.82.hdf5
Epoch 3/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2960 - acc: 0.8995 - val_loss: 0.5708 - val_acc: 0.8191

Epoch 00003: val_acc did not improve from 0.82160
Epoch 4/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2907 - acc: 0.8986 - val_loss: 0.5687 - val_acc: 0.8205

Epoch 00004: val_acc did not improve from 0.82160
Epoch 5/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2871 - acc: 0.9014 - val_loss: 0.5656 - val_acc: 0.8204

Epoch 00005: val_acc did not improve from 0.82160
Epoch 6/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2863 - acc: 0.9012 - val_loss: 0.5689 - val_acc: 0.8209

Epoch 00006: val_acc did not improve from 0.82160
Epoch 7/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2868 - acc: 0.9017 - val_loss: 0.5737 - val_acc: 0.8195

Epoch 00007: val_acc did not improve from 0.82160
Epoch 8/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2849 - acc: 0.9022 - val_loss: 0.5742 - val_acc: 0.8228

Epoch 00008: val_acc improved from 0.82160 to 0.82280, saving model to /content/gdrive/My Drive/Raajesh-08-0.82.hdf5
Epoch 9/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2785 - acc: 0.9039 - val_loss: 0.5666 - val_acc: 0.8242

Epoch 00009: val_acc improved from 0.82280 to 0.82420, saving model to /content/gdrive/My Drive/Raajesh-09-0.82.hdf5
Epoch 10/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2757 - acc: 0.9055 - val_loss: 0.5716 - val_acc: 0.8184

Epoch 00010: val_acc did not improve from 0.82420
Epoch 11/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2765 - acc: 0.9057 - val_loss: 0.5726 - val_acc: 0.8215

Epoch 00011: val_acc did not improve from 0.82420
Epoch 12/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2788 - acc: 0.9036 - val_loss: 0.5778 - val_acc: 0.8204

Epoch 00012: val_acc did not improve from 0.82420
Epoch 13/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2728 - acc: 0.9055 - val_loss: 0.5702 - val_acc: 0.8212

Epoch 00013: val_acc did not improve from 0.82420
Epoch 14/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2667 - acc: 0.9079 - val_loss: 0.5698 - val_acc: 0.8233

Epoch 00014: val_acc did not improve from 0.82420
Epoch 15/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2670 - acc: 0.9067 - val_loss: 0.5763 - val_acc: 0.8209

Epoch 00015: val_acc did not improve from 0.82420
Epoch 16/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2680 - acc: 0.9078 - val_loss: 0.5764 - val_acc: 0.8209

Epoch 00016: val_acc did not improve from 0.82420
Epoch 17/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2683 - acc: 0.9065 - val_loss: 0.5793 - val_acc: 0.8207

Epoch 00017: val_acc did not improve from 0.82420
Epoch 18/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2672 - acc: 0.9066 - val_loss: 0.5776 - val_acc: 0.8209

Epoch 00018: val_acc did not improve from 0.82420
Epoch 19/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2614 - acc: 0.9099 - val_loss: 0.5744 - val_acc: 0.8222

Epoch 00019: val_acc did not improve from 0.82420
Epoch 20/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2616 - acc: 0.9093 - val_loss: 0.5805 - val_acc: 0.8214

Epoch 00020: val_acc did not improve from 0.82420
Epoch 21/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2638 - acc: 0.9085 - val_loss: 0.5762 - val_acc: 0.8218

Epoch 00021: val_acc did not improve from 0.82420
Epoch 22/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2631 - acc: 0.9081 - val_loss: 0.5837 - val_acc: 0.8209

Epoch 00022: val_acc did not improve from 0.82420
Epoch 23/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2599 - acc: 0.9108 - val_loss: 0.5778 - val_acc: 0.8209

Epoch 00023: val_acc did not improve from 0.82420
Epoch 24/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2548 - acc: 0.9103 - val_loss: 0.5811 - val_acc: 0.8194

Epoch 00024: val_acc did not improve from 0.82420
Epoch 25/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2548 - acc: 0.9119 - val_loss: 0.5791 - val_acc: 0.8214

Epoch 00025: val_acc did not improve from 0.82420
Epoch 26/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2565 - acc: 0.9111 - val_loss: 0.5910 - val_acc: 0.8200

Epoch 00026: val_acc did not improve from 0.82420
Epoch 27/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2563 - acc: 0.9113 - val_loss: 0.5854 - val_acc: 0.8213

Epoch 00027: val_acc did not improve from 0.82420
Epoch 28/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2501 - acc: 0.9131 - val_loss: 0.5838 - val_acc: 0.8198

Epoch 00028: val_acc did not improve from 0.82420
Epoch 29/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2473 - acc: 0.9157 - val_loss: 0.5857 - val_acc: 0.8222

Epoch 00029: val_acc did not improve from 0.82420
Epoch 30/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2512 - acc: 0.9123 - val_loss: 0.5894 - val_acc: 0.8185

Epoch 00030: val_acc did not improve from 0.82420
Epoch 31/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2506 - acc: 0.9139 - val_loss: 0.5875 - val_acc: 0.8213

Epoch 00031: val_acc did not improve from 0.82420
Epoch 32/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2467 - acc: 0.9139 - val_loss: 0.5905 - val_acc: 0.8196

Epoch 00032: val_acc did not improve from 0.82420
Epoch 33/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2443 - acc: 0.9149 - val_loss: 0.5865 - val_acc: 0.8200

Epoch 00033: val_acc did not improve from 0.82420
Epoch 34/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2416 - acc: 0.9153 - val_loss: 0.5874 - val_acc: 0.8213

Epoch 00034: val_acc did not improve from 0.82420
Epoch 35/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2458 - acc: 0.9139 - val_loss: 0.5851 - val_acc: 0.8204

Epoch 00035: val_acc did not improve from 0.82420
Epoch 36/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2413 - acc: 0.9165 - val_loss: 0.5897 - val_acc: 0.8205

Epoch 00036: val_acc did not improve from 0.82420
Epoch 37/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2420 - acc: 0.9163 - val_loss: 0.5884 - val_acc: 0.8200

Epoch 00037: val_acc did not improve from 0.82420
Epoch 38/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2384 - acc: 0.9159 - val_loss: 0.5882 - val_acc: 0.8213

Epoch 00038: val_acc did not improve from 0.82420
Epoch 39/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2375 - acc: 0.9163 - val_loss: 0.5919 - val_acc: 0.8219

Epoch 00039: val_acc did not improve from 0.82420
Epoch 40/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2401 - acc: 0.9167 - val_loss: 0.5871 - val_acc: 0.8225

Epoch 00040: val_acc did not improve from 0.82420
Epoch 41/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2416 - acc: 0.9146 - val_loss: 0.5924 - val_acc: 0.8199

Epoch 00041: val_acc did not improve from 0.82420
Epoch 42/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2336 - acc: 0.9190 - val_loss: 0.5932 - val_acc: 0.8210

Epoch 00042: val_acc did not improve from 0.82420
Epoch 43/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2314 - acc: 0.9189 - val_loss: 0.5941 - val_acc: 0.8236

Epoch 00043: val_acc did not improve from 0.82420
Epoch 44/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2339 - acc: 0.9193 - val_loss: 0.6031 - val_acc: 0.8211

Epoch 00044: val_acc did not improve from 0.82420
Epoch 45/50
55000/55000 [==============================] - 200s 4ms/step - loss: 0.2366 - acc: 0.9180 - val_loss: 0.5969 - val_acc: 0.8208

Epoch 00045: val_acc did not improve from 0.82420
Epoch 46/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2307 - acc: 0.9201 - val_loss: 0.5928 - val_acc: 0.8226

Epoch 00046: val_acc did not improve from 0.82420
Epoch 47/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2301 - acc: 0.9189 - val_loss: 0.5949 - val_acc: 0.8227

Epoch 00047: val_acc did not improve from 0.82420
Epoch 48/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2313 - acc: 0.9193 - val_loss: 0.6008 - val_acc: 0.8228

Epoch 00048: val_acc did not improve from 0.82420
Epoch 49/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2287 - acc: 0.9203 - val_loss: 0.6097 - val_acc: 0.8191

Epoch 00049: val_acc did not improve from 0.82420
Epoch 50/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2260 - acc: 0.9209 - val_loss: 0.5995 - val_acc: 0.8224

Epoch 00050: val_acc did not improve from 0.82420
<keras.callbacks.History at 0x7fa957d449b0>

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 55000 samples, validate on 10000 samples
Epoch 1/50
55000/55000 [==============================] - 219s 4ms/step - loss: 0.2858 - acc: 0.9037 - val_loss: 0.5656 - val_acc: 0.8248

Epoch 00001: val_acc improved from -inf to 0.82480, saving model to /content/gdrive/My Drive/Raajesh-01-0.82.hdf5
Epoch 2/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2811 - acc: 0.9039 - val_loss: 0.5666 - val_acc: 0.8241

Epoch 00002: val_acc did not improve from 0.82480
Epoch 3/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2789 - acc: 0.9055 - val_loss: 0.5662 - val_acc: 0.8232

Epoch 00003: val_acc did not improve from 0.82480
Epoch 4/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2792 - acc: 0.9043 - val_loss: 0.5662 - val_acc: 0.8220

Epoch 00004: val_acc did not improve from 0.82480
Epoch 5/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2806 - acc: 0.9043 - val_loss: 0.5653 - val_acc:

Train on 55000 samples, validate on 10000 samples
Epoch 1/50
55000/55000 [==============================] - 219s 4ms/step - loss: 0.2858 - acc: 0.9037 - val_loss: 0.5656 - val_acc: 0.8248

Epoch 00001: val_acc improved from -inf to 0.82480, saving model to /content/gdrive/My Drive/Raajesh-01-0.82.hdf5
Epoch 2/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2811 - acc: 0.9039 - val_loss: 0.5666 - val_acc: 0.8241

Epoch 00002: val_acc did not improve from 0.82480
Epoch 3/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2789 - acc: 0.9055 - val_loss: 0.5662 - val_acc: 0.8232

Epoch 00003: val_acc did not improve from 0.82480
Epoch 4/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2792 - acc: 0.9043 - val_loss: 0.5662 - val_acc: 0.8220

Epoch 00004: val_acc did not improve from 0.82480
Epoch 5/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2806 - acc: 0.9043 - val_loss: 0.5653 - val_acc: 0.8232

Epoch 00005: val_acc did not improve from 0.82480
Epoch 6/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2811 - acc: 0.9038 - val_loss: 0.5656 - val_acc: 0.8229

Epoch 00006: val_acc did not improve from 0.82480
Epoch 7/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2775 - acc: 0.9049 - val_loss: 0.5663 - val_acc: 0.8225

Epoch 00007: val_acc did not improve from 0.82480
Epoch 8/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2756 - acc: 0.9064 - val_loss: 0.5652 - val_acc: 0.8234

Epoch 00008: val_acc did not improve from 0.82480
Epoch 9/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2769 - acc: 0.9061 - val_loss: 0.5660 - val_acc: 0.8233

Epoch 00009: val_acc did not improve from 0.82480
Epoch 10/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2806 - acc: 0.9046 - val_loss: 0.5656 - val_acc: 0.8225

Epoch 00010: val_acc did not improve from 0.82480
Epoch 11/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2744 - acc: 0.9072 - val_loss: 0.5652 - val_acc: 0.8233

Epoch 00011: val_acc did not improve from 0.82480
Epoch 12/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2753 - acc: 0.9060 - val_loss: 0.5660 - val_acc: 0.8220

Epoch 00012: val_acc did not improve from 0.82480
Epoch 13/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2779 - acc: 0.9052 - val_loss: 0.5656 - val_acc: 0.8222

Epoch 00013: val_acc did not improve from 0.82480
Epoch 14/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2749 - acc: 0.9047 - val_loss: 0.5660 - val_acc: 0.8232

Epoch 00014: val_acc did not improve from 0.82480
Epoch 15/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2729 - acc: 0.9064 - val_loss: 0.5660 - val_acc: 0.8227

Epoch 00015: val_acc did not improve from 0.82480
Epoch 16/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2752 - acc: 0.9077 - val_loss: 0.5669 - val_acc: 0.8233

Epoch 00016: val_acc did not improve from 0.82480
Epoch 17/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2727 - acc: 0.9066 - val_loss: 0.5657 - val_acc: 0.8236

Epoch 00017: val_acc did not improve from 0.82480
Epoch 18/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2730 - acc: 0.9067 - val_loss: 0.5660 - val_acc: 0.8224

Epoch 00018: val_acc did not improve from 0.82480
Epoch 19/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2727 - acc: 0.9071 - val_loss: 0.5663 - val_acc: 0.8222

Epoch 00019: val_acc did not improve from 0.82480
Epoch 20/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2714 - acc: 0.9067 - val_loss: 0.5657 - val_acc: 0.8212

Epoch 00020: val_acc did not improve from 0.82480
Epoch 21/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2720 - acc: 0.9075 - val_loss: 0.5671 - val_acc: 0.8223

Epoch 00021: val_acc did not improve from 0.82480
Epoch 22/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2707 - acc: 0.9073 - val_loss: 0.5657 - val_acc: 0.8228

Epoch 00022: val_acc did not improve from 0.82480
Epoch 23/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2727 - acc: 0.9070 - val_loss: 0.5659 - val_acc: 0.8222

Epoch 00023: val_acc did not improve from 0.82480
Epoch 24/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2729 - acc: 0.9060 - val_loss: 0.5667 - val_acc: 0.8226

Epoch 00024: val_acc did not improve from 0.82480
Epoch 25/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2722 - acc: 0.9063 - val_loss: 0.5666 - val_acc: 0.8217

Epoch 00025: val_acc did not improve from 0.82480
Epoch 26/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2712 - acc: 0.9073 - val_loss: 0.5652 - val_acc: 0.8223

Epoch 00026: val_acc did not improve from 0.82480
Epoch 27/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2667 - acc: 0.9083 - val_loss: 0.5660 - val_acc: 0.8221

Epoch 00027: val_acc did not improve from 0.82480
Epoch 28/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2680 - acc: 0.9075 - val_loss: 0.5659 - val_acc: 0.8225

Epoch 00028: val_acc did not improve from 0.82480
Epoch 29/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2710 - acc: 0.9081 - val_loss: 0.5661 - val_acc: 0.8225

Epoch 00029: val_acc did not improve from 0.82480
Epoch 30/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2682 - acc: 0.9083 - val_loss: 0.5663 - val_acc: 0.8227

Epoch 00030: val_acc did not improve from 0.82480
Epoch 31/50
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2736 - acc: 0.9067 - val_loss: 0.5662 - val_acc: 0.8222

Epoch 00031: val_acc did not improve from 0.82480
Epoch 32/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2708 - acc: 0.9064 - val_loss: 0.5667 - val_acc: 0.8230

Epoch 00032: val_acc did not improve from 0.82480
Epoch 33/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2721 - acc: 0.9070 - val_loss: 0.5668 - val_acc: 0.8220

Epoch 00033: val_acc did not improve from 0.82480
Epoch 34/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2687 - acc: 0.9090 - val_loss: 0.5663 - val_acc: 0.8230

Epoch 00034: val_acc did not improve from 0.82480
Epoch 35/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2687 - acc: 0.9071 - val_loss: 0.5667 - val_acc: 0.8223

Epoch 00035: val_acc did not improve from 0.82480
Epoch 36/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2705 - acc: 0.9082 - val_loss: 0.5676 - val_acc: 0.8231

Epoch 00036: val_acc did not improve from 0.82480
Epoch 37/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2698 - acc: 0.9065 - val_loss: 0.5664 - val_acc: 0.8227

Epoch 00037: val_acc did not improve from 0.82480
Epoch 38/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2686 - acc: 0.9091 - val_loss: 0.5664 - val_acc: 0.8229

Epoch 00038: val_acc did not improve from 0.82480
Epoch 39/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2696 - acc: 0.9076 - val_loss: 0.5665 - val_acc: 0.8231

Epoch 00039: val_acc did not improve from 0.82480
Epoch 40/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2695 - acc: 0.9065 - val_loss: 0.5665 - val_acc: 0.8225

Epoch 00040: val_acc did not improve from 0.82480
Epoch 41/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2678 - acc: 0.9087 - val_loss: 0.5679 - val_acc: 0.8230

Epoch 00041: val_acc did not improve from 0.82480
Epoch 42/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2674 - acc: 0.9085 - val_loss: 0.5674 - val_acc: 0.8225

Epoch 00042: val_acc did not improve from 0.82480
Epoch 43/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2626 - acc: 0.9099 - val_loss: 0.5672 - val_acc: 0.8229

Epoch 00043: val_acc did not improve from 0.82480
Epoch 44/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2680 - acc: 0.9073 - val_loss: 0.5673 - val_acc: 0.8227

Epoch 00044: val_acc did not improve from 0.82480
Epoch 45/50
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2672 - acc: 0.9092 - val_loss: 0.5679 - val_acc: 0.8219

Epoch 00045: val_acc did not improve from 0.82480
Epoch 46/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2684 - acc: 0.9085 - val_loss: 0.5677 - val_acc: 0.8226

Epoch 00046: val_acc did not improve from 0.82480
Epoch 47/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2637 - acc: 0.9086 - val_loss: 0.5682 - val_acc: 0.8228

Epoch 00047: val_acc did not improve from 0.82480
Epoch 48/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2676 - acc: 0.9071 - val_loss: 0.5675 - val_acc: 0.8218

Epoch 00048: val_acc did not improve from 0.82480
Epoch 49/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2659 - acc: 0.9095 - val_loss: 0.5694 - val_acc: 0.8228

Epoch 00049: val_acc did not improve from 0.82480
Epoch 50/50
55000/55000 [==============================] - 199s 4ms/step - loss: 0.2611 - acc: 0.9105 - val_loss: 0.5682 - val_acc: 0.8223

Epoch 00050: val_acc did not improve from 0.82480
<keras.callbacks.History at 0x7fc0b130e940>

In [33]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=90,
                    verbose=1,
                    validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 55000 samples, validate on 10000 samples
Epoch 1/90
55000/55000 [==============================] - 219s 4ms/step - loss: 0.2845 - acc: 0.9038 - val_loss: 0.5677 - val_acc: 0.8229

Epoch 00001: val_acc improved from -inf to 0.82290, saving model to /content/gdrive/My Drive/Raajesh-01-0.82.hdf5
Epoch 2/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2810 - acc: 0.9038 - val_loss: 0.5674 - val_acc: 0.8229

Epoch 00002: val_acc did not improve from 0.82290
Epoch 3/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2794 - acc: 0.9048 - val_loss: 0.5663 - val_acc: 0.8222

Epoch 00003: val_acc did not improve from 0.82290
Epoch 4/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2791 - acc: 0.9034 - val_loss: 0.5697 - val_acc: 0.8217

Epoch 00004: val_acc did not improve from 0.82290
Epoch 5/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2802 - acc: 0.9039 - val_loss: 0.5699 - val_acc:

Train on 55000 samples, validate on 10000 samples
Epoch 1/90
55000/55000 [==============================] - 219s 4ms/step - loss: 0.2845 - acc: 0.9038 - val_loss: 0.5677 - val_acc: 0.8229

Epoch 00001: val_acc improved from -inf to 0.82290, saving model to /content/gdrive/My Drive/Raajesh-01-0.82.hdf5
Epoch 2/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2810 - acc: 0.9038 - val_loss: 0.5674 - val_acc: 0.8229

Epoch 00002: val_acc did not improve from 0.82290
Epoch 3/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2794 - acc: 0.9048 - val_loss: 0.5663 - val_acc: 0.8222

Epoch 00003: val_acc did not improve from 0.82290
Epoch 4/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2791 - acc: 0.9034 - val_loss: 0.5697 - val_acc: 0.8217

Epoch 00004: val_acc did not improve from 0.82290
Epoch 5/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2802 - acc: 0.9039 - val_loss: 0.5699 - val_acc: 0.8222

Epoch 00005: val_acc did not improve from 0.82290
Epoch 6/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2782 - acc: 0.9041 - val_loss: 0.5672 - val_acc: 0.8226

Epoch 00006: val_acc did not improve from 0.82290
Epoch 7/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2750 - acc: 0.9062 - val_loss: 0.5671 - val_acc: 0.8217

Epoch 00007: val_acc did not improve from 0.82290
Epoch 8/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2777 - acc: 0.9047 - val_loss: 0.5669 - val_acc: 0.8214

Epoch 00008: val_acc did not improve from 0.82290
Epoch 9/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2770 - acc: 0.9061 - val_loss: 0.5691 - val_acc: 0.8215

Epoch 00009: val_acc did not improve from 0.82290
Epoch 10/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2789 - acc: 0.9045 - val_loss: 0.5728 - val_acc: 0.8208

Epoch 00010: val_acc did not improve from 0.82290
Epoch 11/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2754 - acc: 0.9054 - val_loss: 0.5721 - val_acc: 0.8205

Epoch 00011: val_acc did not improve from 0.82290
Epoch 12/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2722 - acc: 0.9067 - val_loss: 0.5704 - val_acc: 0.8209

Epoch 00012: val_acc did not improve from 0.82290
Epoch 13/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2759 - acc: 0.9051 - val_loss: 0.5701 - val_acc: 0.8215

Epoch 00013: val_acc did not improve from 0.82290
Epoch 14/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2748 - acc: 0.9045 - val_loss: 0.5729 - val_acc: 0.8213

Epoch 00014: val_acc did not improve from 0.82290
Epoch 15/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2713 - acc: 0.9071 - val_loss: 0.5712 - val_acc: 0.8206

Epoch 00015: val_acc did not improve from 0.82290
Epoch 16/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2694 - acc: 0.9075 - val_loss: 0.5710 - val_acc: 0.8219

Epoch 00016: val_acc did not improve from 0.82290
Epoch 17/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2665 - acc: 0.9077 - val_loss: 0.5707 - val_acc: 0.8217

Epoch 00017: val_acc did not improve from 0.82290
Epoch 18/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2683 - acc: 0.9066 - val_loss: 0.5723 - val_acc: 0.8211

Epoch 00018: val_acc did not improve from 0.82290
Epoch 19/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2718 - acc: 0.9066 - val_loss: 0.5736 - val_acc: 0.8206

Epoch 00019: val_acc did not improve from 0.82290
Epoch 20/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2726 - acc: 0.9063 - val_loss: 0.5732 - val_acc: 0.8201

Epoch 00020: val_acc did not improve from 0.82290
Epoch 21/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2657 - acc: 0.9089 - val_loss: 0.5717 - val_acc: 0.8218

Epoch 00021: val_acc did not improve from 0.82290
Epoch 22/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2660 - acc: 0.9085 - val_loss: 0.5726 - val_acc: 0.8221

Epoch 00022: val_acc did not improve from 0.82290
Epoch 23/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2649 - acc: 0.9081 - val_loss: 0.5736 - val_acc: 0.8219

Epoch 00023: val_acc did not improve from 0.82290
Epoch 24/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2690 - acc: 0.9070 - val_loss: 0.5726 - val_acc: 0.8219

Epoch 00024: val_acc did not improve from 0.82290
Epoch 25/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2627 - acc: 0.9105 - val_loss: 0.5721 - val_acc: 0.8217

Epoch 00025: val_acc did not improve from 0.82290
Epoch 26/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2645 - acc: 0.9091 - val_loss: 0.5731 - val_acc: 0.8205

Epoch 00026: val_acc did not improve from 0.82290
Epoch 27/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2650 - acc: 0.9084 - val_loss: 0.5730 - val_acc: 0.8224

Epoch 00027: val_acc did not improve from 0.82290
Epoch 28/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2670 - acc: 0.9083 - val_loss: 0.5755 - val_acc: 0.8211

Epoch 00028: val_acc did not improve from 0.82290
Epoch 29/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2647 - acc: 0.9083 - val_loss: 0.5749 - val_acc: 0.8222

Epoch 00029: val_acc did not improve from 0.82290
Epoch 30/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2620 - acc: 0.9100 - val_loss: 0.5751 - val_acc: 0.8235

Epoch 00030: val_acc improved from 0.82290 to 0.82350, saving model to /content/gdrive/My Drive/Raajesh-30-0.82.hdf5
Epoch 31/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2601 - acc: 0.9100 - val_loss: 0.5747 - val_acc: 0.8241

Epoch 00031: val_acc improved from 0.82350 to 0.82410, saving model to /content/gdrive/My Drive/Raajesh-31-0.82.hdf5
Epoch 32/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2618 - acc: 0.9101 - val_loss: 0.5779 - val_acc: 0.8214

Epoch 00032: val_acc did not improve from 0.82410
Epoch 33/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2647 - acc: 0.9078 - val_loss: 0.5756 - val_acc: 0.8218

Epoch 00033: val_acc did not improve from 0.82410
Epoch 34/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2594 - acc: 0.9097 - val_loss: 0.5763 - val_acc: 0.8236

Epoch 00034: val_acc did not improve from 0.82410
Epoch 35/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2589 - acc: 0.9110 - val_loss: 0.5757 - val_acc: 0.8235

Epoch 00035: val_acc did not improve from 0.82410
Epoch 36/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2573 - acc: 0.9107 - val_loss: 0.5754 - val_acc: 0.8220

Epoch 00036: val_acc did not improve from 0.82410
Epoch 37/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2575 - acc: 0.9111 - val_loss: 0.5760 - val_acc: 0.8230

Epoch 00037: val_acc did not improve from 0.82410
Epoch 38/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2627 - acc: 0.9098 - val_loss: 0.5758 - val_acc: 0.8222

Epoch 00038: val_acc did not improve from 0.82410
Epoch 39/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2579 - acc: 0.9097 - val_loss: 0.5769 - val_acc: 0.8219

Epoch 00039: val_acc did not improve from 0.82410
Epoch 40/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2558 - acc: 0.9116 - val_loss: 0.5772 - val_acc: 0.8229

Epoch 00040: val_acc did not improve from 0.82410
Epoch 41/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2572 - acc: 0.9106 - val_loss: 0.5790 - val_acc: 0.8219

Epoch 00041: val_acc did not improve from 0.82410
Epoch 42/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2575 - acc: 0.9101 - val_loss: 0.5792 - val_acc: 0.8208

Epoch 00042: val_acc did not improve from 0.82410
Epoch 43/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2559 - acc: 0.9105 - val_loss: 0.5776 - val_acc: 0.8231

Epoch 00043: val_acc did not improve from 0.82410
Epoch 44/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2564 - acc: 0.9116 - val_loss: 0.5781 - val_acc: 0.8234

Epoch 00044: val_acc did not improve from 0.82410
Epoch 45/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2565 - acc: 0.9115 - val_loss: 0.5786 - val_acc: 0.8220

Epoch 00045: val_acc did not improve from 0.82410
Epoch 46/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2545 - acc: 0.9129 - val_loss: 0.5814 - val_acc: 0.8219

Epoch 00046: val_acc did not improve from 0.82410
Epoch 47/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2582 - acc: 0.9103 - val_loss: 0.5805 - val_acc: 0.8232

Epoch 00047: val_acc did not improve from 0.82410
Epoch 48/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2572 - acc: 0.9111 - val_loss: 0.5793 - val_acc: 0.8226

Epoch 00048: val_acc did not improve from 0.82410
Epoch 49/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2551 - acc: 0.9119 - val_loss: 0.5788 - val_acc: 0.8228

Epoch 00049: val_acc did not improve from 0.82410
Epoch 50/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2557 - acc: 0.9118 - val_loss: 0.5804 - val_acc: 0.8212

Epoch 00050: val_acc did not improve from 0.82410
Epoch 51/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2549 - acc: 0.9116 - val_loss: 0.5787 - val_acc: 0.8221

Epoch 00051: val_acc did not improve from 0.82410
Epoch 52/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2568 - acc: 0.9119 - val_loss: 0.5799 - val_acc: 0.8228

Epoch 00052: val_acc did not improve from 0.82410
Epoch 53/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2524 - acc: 0.9128 - val_loss: 0.5802 - val_acc: 0.8231

Epoch 00053: val_acc did not improve from 0.82410
Epoch 54/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2497 - acc: 0.9131 - val_loss: 0.5795 - val_acc: 0.8228

Epoch 00054: val_acc did not improve from 0.82410
Epoch 55/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2512 - acc: 0.9128 - val_loss: 0.5811 - val_acc: 0.8233

Epoch 00055: val_acc did not improve from 0.82410
Epoch 56/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2503 - acc: 0.9131 - val_loss: 0.5804 - val_acc: 0.8222

Epoch 00056: val_acc did not improve from 0.82410
Epoch 57/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2494 - acc: 0.9139 - val_loss: 0.5808 - val_acc: 0.8218

Epoch 00057: val_acc did not improve from 0.82410
Epoch 58/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2479 - acc: 0.9152 - val_loss: 0.5800 - val_acc: 0.8224

Epoch 00058: val_acc did not improve from 0.82410
Epoch 59/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2492 - acc: 0.9153 - val_loss: 0.5788 - val_acc: 0.8229

Epoch 00059: val_acc did not improve from 0.82410
Epoch 60/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2474 - acc: 0.9139 - val_loss: 0.5826 - val_acc: 0.8214

Epoch 00060: val_acc did not improve from 0.82410
Epoch 61/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2494 - acc: 0.9136 - val_loss: 0.5816 - val_acc: 0.8235

Epoch 00061: val_acc did not improve from 0.82410
Epoch 62/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2498 - acc: 0.9130 - val_loss: 0.5813 - val_acc: 0.8232

Epoch 00062: val_acc did not improve from 0.82410
Epoch 63/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2465 - acc: 0.9150 - val_loss: 0.5813 - val_acc: 0.8225

Epoch 00063: val_acc did not improve from 0.82410
Epoch 64/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2432 - acc: 0.9165 - val_loss: 0.5812 - val_acc: 0.8223

Epoch 00064: val_acc did not improve from 0.82410
Epoch 65/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2507 - acc: 0.9123 - val_loss: 0.5838 - val_acc: 0.8213

Epoch 00065: val_acc did not improve from 0.82410
Epoch 66/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2464 - acc: 0.9142 - val_loss: 0.5824 - val_acc: 0.8213

Epoch 00066: val_acc did not improve from 0.82410
Epoch 67/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2460 - acc: 0.9145 - val_loss: 0.5827 - val_acc: 0.8220

Epoch 00067: val_acc did not improve from 0.82410
Epoch 68/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2422 - acc: 0.9160 - val_loss: 0.5817 - val_acc: 0.8220

Epoch 00068: val_acc did not improve from 0.82410
Epoch 69/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2421 - acc: 0.9168 - val_loss: 0.5828 - val_acc: 0.8229

Epoch 00069: val_acc did not improve from 0.82410
Epoch 70/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2432 - acc: 0.9147 - val_loss: 0.5849 - val_acc: 0.8207

Epoch 00070: val_acc did not improve from 0.82410
Epoch 71/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2463 - acc: 0.9146 - val_loss: 0.5854 - val_acc: 0.8232

Epoch 00071: val_acc did not improve from 0.82410
Epoch 72/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2421 - acc: 0.9143 - val_loss: 0.5839 - val_acc: 0.8218

Epoch 00072: val_acc did not improve from 0.82410
Epoch 73/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2432 - acc: 0.9165 - val_loss: 0.5839 - val_acc: 0.8223

Epoch 00073: val_acc did not improve from 0.82410
Epoch 74/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2414 - acc: 0.9160 - val_loss: 0.5846 - val_acc: 0.8213

Epoch 00074: val_acc did not improve from 0.82410
Epoch 75/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2417 - acc: 0.9153 - val_loss: 0.5859 - val_acc: 0.8196

Epoch 00075: val_acc did not improve from 0.82410
Epoch 76/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2439 - acc: 0.9157 - val_loss: 0.5835 - val_acc: 0.8224

Epoch 00076: val_acc did not improve from 0.82410
Epoch 77/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2373 - acc: 0.9172 - val_loss: 0.5844 - val_acc: 0.8210

Epoch 00077: val_acc did not improve from 0.82410
Epoch 78/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2405 - acc: 0.9162 - val_loss: 0.5844 - val_acc: 0.8198

Epoch 00078: val_acc did not improve from 0.82410
Epoch 79/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2403 - acc: 0.9164 - val_loss: 0.5880 - val_acc: 0.8212

Epoch 00079: val_acc did not improve from 0.82410
Epoch 80/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2386 - acc: 0.9175 - val_loss: 0.5887 - val_acc: 0.8207

Epoch 00080: val_acc did not improve from 0.82410
Epoch 81/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2386 - acc: 0.9178 - val_loss: 0.5868 - val_acc: 0.8212

Epoch 00081: val_acc did not improve from 0.82410
Epoch 82/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2359 - acc: 0.9180 - val_loss: 0.5868 - val_acc: 0.8210

Epoch 00082: val_acc did not improve from 0.82410
Epoch 83/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2401 - acc: 0.9168 - val_loss: 0.5869 - val_acc: 0.8206

Epoch 00083: val_acc did not improve from 0.82410
Epoch 84/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2402 - acc: 0.9160 - val_loss: 0.5903 - val_acc: 0.8211

Epoch 00084: val_acc did not improve from 0.82410
Epoch 85/90
55000/55000 [==============================] - 198s 4ms/step - loss: 0.2369 - acc: 0.9182 - val_loss: 0.5873 - val_acc: 0.8206

Epoch 00085: val_acc did not improve from 0.82410
Epoch 86/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2339 - acc: 0.9190 - val_loss: 0.5866 - val_acc: 0.8210

Epoch 00086: val_acc did not improve from 0.82410
Epoch 87/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2355 - acc: 0.9181 - val_loss: 0.5875 - val_acc: 0.8216

Epoch 00087: val_acc did not improve from 0.82410
Epoch 88/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2372 - acc: 0.9191 - val_loss: 0.5890 - val_acc: 0.8206

Epoch 00088: val_acc did not improve from 0.82410
Epoch 89/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2362 - acc: 0.9183 - val_loss: 0.5880 - val_acc: 0.8185

Epoch 00089: val_acc did not improve from 0.82410
Epoch 90/90
55000/55000 [==============================] - 197s 4ms/step - loss: 0.2384 - acc: 0.9166 - val_loss: 0.5888 - val_acc: 0.8198

Epoch 00090: val_acc did not improve from 0.82410
<keras.callbacks.History at 0x7f5399a8fc18>

In [0]:
model.load_weights('/content/gdrive/My Drive/Raajesh-10-0.70.hdf5')

In [0]:
model.load_weights('/content/gdrive/My Drive/Raajesh-42-0.81.hdf5')

In [0]:
model.load_weights('/content/gdrive/My Drive/Raajesh-09-0.82.hdf5')

In [0]:
model.load_weights('/content/gdrive/My Drive/Raajesh-01-0.82.hdf5')

In [34]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 17s 2ms/step
Test loss: 0.588770724606514
Test accuracy: 0.8198
